In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import os

In [ ]:
#----------TRAIN-------------#
target_path = '../data/chisato/goal/'
contents = os.listdir(target_path)
data = pd.DataFrame()
num = 0
for c in range(len(contents)):
    #print(target_path+contents[c])
    n_data = pd.read_csv(target_path+contents[c],header=None)
    data = pd.concat([data,n_data])
plt.hold(True)
plt.grid(color='gray')
plt.scatter(data[1],data[0], s=10, c='blue',label='TrainData')
    
#----------TEST-------------#
test_path = '../data/seki/log201805261718.csv'
contents = os.listdir(target_path)
test_data = pd.read_csv(test_path,header=None)
plt.grid(color='gray')
plt.scatter(test_data[1],test_data[0], s=10, c='red',label='TestData')
plt.legend()

In [ ]:
IN_size = 2
TAR_size = 2
INPUT = tf.placeholder(shape=[None, IN_size], dtype=tf.float32)
TARGET = tf.placeholder(shape=[None, TAR_size], dtype=tf.float32)

hidden_size = [25]
batch_size = 100
iter_size = 50000

#　隠れ層の作成 （last layerに直結）
weights = []
biases = []
layers = []
tmp_size = IN_size
last_layer = INPUT
for hsize in hidden_size:
    weights.append(tf.Variable(tf.random_normal(shape=[tmp_size, hsize], stddev=10.0)))
    biases.append(tf.Variable(tf.random_normal(shape=[hsize], stddev=10.0)))
    layers.append(tf.nn.relu(tf.add(tf.matmul(last_layer, weights[-1]), biases[-1]))) # 活性化関数
    tmp_size = hsize
    last_layer = layers[-1]
    print(layers[-1])
#　出力層
weights.append(tf.Variable(tf.random_normal(shape=[tmp_size, TAR_size], stddev=10.0)))
biases.append(tf.Variable(tf.random_normal(shape=[TAR_size], stddev=10.0)))
layers.append(tf.add(tf.matmul(last_layer, weights[-1]), biases[-1]))
final_output = layers[-1]

loss = tf.losses.mean_squared_error(TARGET,final_output)

In [ ]:
# 重みのrestore
saver = tf.train.Saver()
predicts_sess = tf.Session()
ckpt = tf.train.get_checkpoint_state('./')
if ckpt == None:
    print('!!!!!!!WEIGHTS NOT FOUND!!!!!!')

saver.restore(predicts_sess, './model.ckpt')

In [ ]:
test_data.columns = ['bx','by','jx','jy']
test_data = test_data[test_data.bx!=-1]
s_INPUT = np.array(test_data.iloc[:,0:2])
print(s_INPUT.shape)
s_ANS = np.array(test_data.iloc[:,2:4])
print(s_ANS.shape)
pred = np.empty((0,3),np.float32)
for i in range(len(s_INPUT)):
    s_input = np.array([s_INPUT[i]])/100
    s_ans = np.array([s_ANS[i]])
    #s_ans = np.array([s_ANS[i]])/100
    res = predicts_sess.run(final_output, feed_dict={INPUT: s_input})
    test_loss = predicts_sess.run(loss, feed_dict={INPUT: s_input,TARGET:s_ans})
    #print(np.c_[test_loss, s_input])
    col = np.c_[test_loss, s_input]
    pred = np.append(pred, col, axis=0)
    
p_pred = pd.DataFrame(pred,columns=list('pxy'))
sorted_data = p_pred.sort_values(by='p',ascending=False) #p列において値の大きい順にソート
print(len(pred))

In [ ]:
plt.hold(True)
plt.scatter(data[1]/100,data[0]/100, s=10, c='palegreen',label='TrainData')

thresh = 1

#----------TEST-------------#
for t in range(len(pred)):
    if pred[t][0] < thresh: #まあほぼ正解
        plt.scatter(pred[t][2],pred[t][1], s=10, c='lightcoral')
    else: #失敗
        plt.scatter(pred[t][2],pred[t][1], s=10, c='c')
        
